# SPI Flash

* https://forum.micropython.org/viewtopic.php?f=12&t=9772&hilit=Ttmetro
* https://github.com/peterhinch/micropython_eeprom/tree/master/flash

| Function  | Pin  |
| --------- |------|
| SCK       | PB3  | 
| MOSI      | PB5  | 
| MISO      | PB4  | 
| CS        | PA15 | 

In [1]:
%register 'serial:///dev/serial0'
%discover
%connect robot-stm
%rlist

robot-stm  serial:///dev/serial0  
Connected to robot-stm @ serial:///dev/serial0
                            flash/
    372  Mar 18 19:27 2021      boot.py
                                lib/
   6644  Mar 19 13:29 2021          bdevice.py
   6731  Mar 19 13:29 2021          flash_spi.py
                            spi/
                                lib/
   6491  Mar 19 13:48 2021          bno055.py
   5598  Mar 19 13:48 2021          bno055_base.py
   1078  Mar 19 13:48 2021          bno055_test.py
   2073  Mar 19 13:48 2021          encoder.py
    244  Mar 19 13:48 2021          init_urpc_client.py
   2094  Mar 19 13:48 2021          logging.py
   1174  Mar 19 13:48 2021          neopixel.py
   1091  Mar 19 13:48 2021          tb6612.py
    317  Mar 19 13:48 2021      secrets.py


## Optional: convert `/flash` to `little_fs`

**ATTENTION:** Erases '/flash'

In [1]:
import os

os.umount('/flash')
os.VfsLfs2.mkfs(pyb.Flash(start=0))
os.mount(pyb.Flash(start=0), '/flash')
os.chdir('/flash')
print(os.statvfs('/flash'))

(512, 512, 224, 222, 222, 0, 0, 0, 0, 255)


## First time only: create `/spi` filesystem

**ATTENTION:** Erases '/spi'

In [1]:
# load software

%connect robot-stm
%rdiff
print("flashing ...")
%rsync

Connected to robot-stm @ serial:///dev/ttyACM0
Directories match
flashing ...
Directories match


In [1]:
%rlist

                            flash/
    372  Mar 18 19:27 2021      boot.py
                                lib/
   6644  Mar 19 13:29 2021          bdevice.py
   6731  Mar 19 13:29 2021          flash_spi.py
                            spi/


In [1]:
# access flash chip

from machine import SPI, Pin
from flash_spi import FLASH
import os

spi = SPI(1, baudrate=10_000_000, polarity=1, phase=1)
cspins = (Pin('FLASH_CS', Pin.OUT, value=1),)
flash = FLASH(spi, cspins)

# create filesystem

os.VfsLfs2.mkfs(flash)

# mount /spi

os.mount(flash, "/spi")

1 chips detected. Total flash size 2MiB.


Traceback (most recent call last):
  File "<stdin>", line 17, in <module>
OSError: [Errno 1] EPERM



In [1]:
import os
for r in ['/flash', '/spi']:
    s = os.statvfs(r)
    print("{:6} kB     ".format(int(s[1]*s[3]/1024)), s)

    94 kB      (512, 512, 224, 189, 189, 0, 0, 0, 0, 255)
  2047 kB      (512, 512, 4096, 4094, 4094, 0, 0, 0, 0, 255)


In [1]:
%rlist

                            flash/
    372  Mar 18 19:27 2021      boot.py
                                lib/
   6644  Mar 19 13:29 2021          bdevice.py
   6731  Mar 19 13:29 2021          flash_spi.py
                            spi/


In [1]:
%rdiff

ADD     /spi/lib/bno055.py
ADD     /spi/lib/bno055_base.py
ADD     /spi/lib/bno055_test.py
ADD     /spi/lib/encoder.py
ADD     /spi/lib/init_urpc_client.py
ADD     /spi/lib/logging.py
ADD     /spi/lib/neopixel.py
ADD     /spi/lib/tb6612.py
ADD     /spi/secrets.py


In [1]:
%rsync

ADD     /spi/lib/bno055.py
ADD     /spi/lib/bno055_base.py
ADD     /spi/lib/bno055_test.py
ADD     /spi/lib/encoder.py
ADD     /spi/lib/init_urpc_client.py
ADD     /spi/lib/logging.py
ADD     /spi/lib/neopixel.py
ADD     /spi/lib/tb6612.py
ADD     /spi/secrets.py
